In [24]:
import pandas as pd
import altair as alt
import plotly.express as px
import numpy as np
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

## Part 1

In [27]:
dwh = pd.read_csv("1 - Popularite vs temps/data_part_1.csv")
dwh

,prenom,year,nombre,parts
0,MARIE,1900,49752,12.740426
1,MARIE,1901,53177,12.405461
2,MARIE,1902,52889,11.997051
3,MARIE,1903,51443,11.595977
4,MARIE,1904,51075,11.257067
...,...,...,...,...
24195,LILIANE,2016,8,0.001496
24196,LILIANE,2017,9,0.001729
24197,LILIANE,2018,16,0.003133
24198,LILIANE,2019,10,0.001985


In [ ]:
brush = alt.selection_interval()

birth_count = alt.Chart(dwh, title="Nombre de naissances par année").mark_bar().encode(
    alt.X("year:T"),
    y='sum(nombre)',
).properties(width=1100, height=100).add_selection(brush)

alt.data_transformers.enable('default', max_rows=None)
names_hist = alt.Chart(dwh, title="Naissances par prénom").mark_bar().encode(
    alt.Y("prenom:N", sort="-x"),
    alt.X('sum(nombre)'),
).transform_window(
    rank='rank(x)',
    sort=[alt.SortField('x', order='descending')]
).transform_filter(
    brush & (alt.datum.nombre > 10_000)
).properties(width=1000, height=500)

hist = alt.Chart(dwh, title="").mark_bar().encode(
    alt.X("parts:Q", bin=True),
    y='count()',
).properties(width=1100, height=100).add_selection(brush).transform_filter(brush)

names_hist & birth_count & hist

alt.VConcatChart(...)

## Part 2

In [28]:
def df_names_fr():
    df = pd.read_csv("data/dpt2020.csv",sep=';',keep_default_na=False)
    df['sexe'] = df['sexe'].apply(lambda x: 'M' if (x==1) else 'F')
    df = df[['annais', 'preusuel','dpt', 'sexe', 'nombre']]
    df = df.rename(columns={"annais": "year", "preusuel": "name","dpt": "department", "sexe":"gender", "nombre":"births"})
    df = df.sort_values(by = ['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    df = df[df['year'] != 'XXXX']
    df = df[df['name'] != '_PRENOMS_RARES']
    df = df[df['name'].apply(lambda x: len(str(x)) > 1)]
    df = df[df['name'].apply(lambda x: (isinstance(x,str)))]
    df['name'] = df['name'].apply(lambda x: x.capitalize())
    df['name'] = df['name'].str.replace('-','')
    df = df.reset_index()
    df = df.drop(columns=['index'])
    return df

In [6]:
df = df_names_fr()

In [7]:
df.head()

,year,name,department,gender,births
0,1900,Marie,29,F,2519
1,1900,Marie,75,F,1576
2,1900,Suzanne,75,F,1382
3,1900,Marie,56,F,1358
4,1900,Marie,59,F,1353


In [8]:
df_agg = df.groupby('name').sum()

/tmp/ipykernel_682/2308446999.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agg = df.groupby('name').sum()


In [9]:
df_agg.head()

,births
name,
Aadil,6
Aahil,3
Aaliya,3
Aaliyah,1794
Aalya,6


In [10]:
df_agg = df_agg.sort_values(by=['births'], ascending = False)

In [11]:
df_agg_100 = df_agg.head(100)

In [12]:
#list(df_agg_100.index.values)
df_reduced = df.loc[df['name'].isin(list(df_agg_100.index.values))]

In [13]:
df_zizou = df.loc[df['name'] == 'Eloïse']

In [14]:
#Question 1
# input_dropdown = alt.binding_select(options=df_reduced['department'].unique() + [None], name='Department ')

input_dropdown = alt.binding_select(
    # Add the empty selection which shows all when clicked
    options=list(sorted(df_reduced['department'].unique())) + [None],
    labels=list(sorted(df_reduced['department'].unique())) + ['Tous'],
    name='Département: '
)

selection = alt.selection_point(fields=['department'], bind=input_dropdown)

# color = alt.condition(
#     selection,
#     alt.Color('name:N'),
#     alt.value('lightgray')
# )

alt.Chart( df_reduced, title="Répartition des prénoms en fonction des départements" ).mark_line().encode(
    y = alt.Y('sum(births):Q', axis=alt.Axis(title='Nombre de naissances')),
    x = alt.X('year:Q', axis=alt.Axis(title='Année')),
    color=alt.Color('name:N', title='Prénom'),
    tooltip=alt.Tooltip('name:N', title='Prénom')
).add_params(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

In [16]:
df_zizou = df.loc[df['name'] == 'Marie']
input_dropdown = alt.binding_select(
    # Add the empty selection which shows all when clicked
    options=list(sorted(df_zizou['department'].unique())) + [None],
    labels=list(sorted(df_zizou['department'].unique())) + ['Tous'],
    name='Département: '
)

selection = alt.selection_point(fields=['department'], bind=input_dropdown)

# color = alt.condition(
#     selection,
#     alt.Color('name:N'),
#     alt.value('lightgray')
# )

alt.Chart( df_zizou, title="Répartition des prénoms en fonction des départements" ).mark_line().encode(
    y = alt.Y('sum(births):Q', axis=alt.Axis(title='Nombre de naissances')),
    x = alt.X('year:Q', axis=alt.Axis(title='Année')),
    color=alt.Color('name:N', title='Prénom'),
    tooltip=alt.Tooltip('name:N', title='Prénom')
).add_params(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

## Part 3

In [17]:
df_agg_gen = df.groupby(['name', 'gender']).sum()
df_agg_gen.head()

/tmp/ipykernel_682/3109333759.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agg_gen = df.groupby(['name', 'gender']).sum()


,,births
name,gender,
Aadil,M,6
Aahil,M,3
Aaliya,F,3
Aaliyah,F,1794
Aalya,F,6


In [18]:
len(df_agg_gen.loc[df_agg_gen['births']>=100])

4959

In [19]:
df_agg_gen_100 = df_agg_gen.loc[df_agg_gen['births']>=100]

In [20]:
sex_flipped = {'M':'F','F':'M'}
F_ratio = []
birth_sum = []
for idx in df_agg_gen_100.index:
    name = idx[0]
    sex = idx[1]
    birth = df_agg_gen_100.loc[idx]['births']
    flipped = sex_flipped[sex]
    try:
        birth_flipped = df_agg_gen_100.loc[(name, flipped)]['births']
    except:
        birth_flipped = 0
    if sex == 'M':
        F_ratio.append((birth_flipped)/(birth+birth_flipped))
    else:
        F_ratio.append((birth)/(birth+birth_flipped))
    birth_sum.append(birth+birth_flipped)





In [21]:
df_agg_gen_100['F_ratio'] = F_ratio
df_agg_gen_100.head()

/tmp/ipykernel_682/1465310909.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg_gen_100['F_ratio'] = F_ratio


,,births,F_ratio
name,gender,,
Aaliyah,F,1794,1.0
Aaron,M,28278,0.0
Abby,F,1157,1.0
Abd,M,581,0.0
Abdallah,M,4180,0.0


In [22]:
df_agg_gen_100['birth_sum'] = birth_sum
df_agg_gen_100.head()

/tmp/ipykernel_682/2677115197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg_gen_100['birth_sum'] = birth_sum


,,births,F_ratio,birth_sum
name,gender,,,
Aaliyah,F,1794,1.0,1794
Aaron,M,28278,0.0,28278
Abby,F,1157,1.0,1157
Abd,M,581,0.0,581
Abdallah,M,4180,0.0,4180


In [23]:
# input_dropdown = alt.binding_select(
#     # Add the empty selection which shows all when clicked
#     options=list(df_reduced['department'].unique()) + [None],
#     labels=list(df_reduced['department'].unique()) + ['All'],
#     name='Department: '
# )

# selection = alt.selection_point(fields=['department'], bind=input_dropdown)

# color = alt.condition(
#     selection,
#     alt.Color('name:N'),
#     alt.value('lightgray')
# )

alt.Chart( df_agg_gen_100.reset_index(), title="Répartition des prénoms par sexe" ).mark_circle(size=60).encode(
    y = alt.Y('birth_sum:Q', axis=alt.Axis(title='Nombre de naissances')).scale(type="log"),
    x = alt.X('F_ratio:Q', axis=alt.Axis(title='Ratio du prénom donné à des filles')),
    #color='F_ratio:Q',
    color=alt.Color('F_ratio:Q', scale=alt.Scale(range=['#386cb0','purple', '#f0027f']), title='Ratio'),
    tooltip=[alt.Tooltip('name:N', title='Prénom'), alt.Tooltip('birth_sum:Q', title='Nombre de naissances')]
)
# ).add_params(
#     selection
# ).transform_filter(
#     selection
# )

alt.Chart(...)